<a href="https://colab.research.google.com/github/dsynderg/479_Final_Project/blob/main/ModelUser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install OpenNMT-py
!pip install "numpy<2.0" # This fixes an error caused by OpenNMT not being maintained
import random

In [1]:
modelname = "modle_step_180000.pt"

In [2]:
outputname = "2000_check"

In [3]:
language = "complete"

In [4]:
translate_yaml_text = f"""
model: /content/{modelname}
src: /content/src_test.txt
output: /content/{outputname}
gpu: 0
verbose: True

# Sentencepiece Params

# Tokenization options
transforms: onmt_tokenize
src_subword_type: sentencepiece
src_subword_model: english_sp_model.model
tgt_subword_type: sentencepiece
tgt_subword_model: {language}_sp_model.model

src_subword_nbest: 20
src_subword_alpha: 0.1
"""

with open("translate_yaml.yaml", "w") as f:
    f.write(translate_yaml_text)

In [ ]:
!onmt_translate -config translate_yaml.yaml

In [6]:
!python -m pip install sacrebleu

In [7]:
from sacrebleu import corpus_bleu

with open("/content/tgt_test.txt", encoding="utf-8") as f:
    refs = [r.strip() for r in f]

with open(outputname, encoding="utf-8") as f:
    hyps = [h.strip() for h in f]

assert len(hyps) == len(refs), "Files must have same number of lines"

score = corpus_bleu(hyps, [refs])  # references must be a list-of-lists
print(f"BLEU = {score.score:.2f}, details: {score.format()}")

BLEU = 36.25, details: BLEU = 36.25 60.5/41.2/30.3/22.9 (BP = 1.000 ratio = 1.113 hyp_len = 45671 ref_len = 41019)


In [ ]:
!python -m pip install sacrebleu

In [8]:
# example: compute_chrf.py
from sacrebleu import corpus_chrf

with open("tgt_test.txt", encoding="utf-8") as f:
    refs = [line.strip() for line in f if line.strip()]

with open(outputname, encoding="utf-8") as f:
    hyps = [line.strip() for line in f if line.strip()]

assert len(hyps) == len(refs), "Files must have same number of lines"

# corpus_chrf expects hyps, [refs]; beta controls precision/recall tradeoff (default 2.0)
score = corpus_chrf(hyps, [refs], beta=2.0)
print(f"chrF = {score.score:.4f}")

chrF = 59.7886
